In [1]:
%matplotlib inline
import pymortar
import pytz
import pandas as pd
import time
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn import linear_model, preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error, r2_score

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
client = pymortar.Client({
    'mortar_address': 'api.mortardata.org',
    'username': "johnyang",
    'password': "printfhdmi1cloudmarker&L",
})

INFO:root:Got event ChannelConnectivity.IDLE
INFO:root:loaded .pymortartoken.json token


In [3]:
meterview = pymortar.View(
    name='meter',
    definition='''SELECT ?meter WHERE { 
    ?meter rdf:type brick:Building_Electric_Meter .
    };'''
)
equipview = pymortar.View(
    name='equip',
    definition='''SELECT ?equip ?state WHERE {
        ?equip rdf:type/rdfs:subClassOf* brick:Equipment .
        ?equip bf:hasPoint ?state .
        ?state rdf:type/rdfs:subClassOf* brick:Status
    };'''
)

meterdf = pymortar.DataFrame(
    name='meter',
    aggregation=pymortar.MEAN,
    window='15m',
    timeseries=[
        pymortar.Timeseries(
            view='meter',
            dataVars=['?meter']
        )
    ]
)

equipdf = pymortar.DataFrame(
    name='equip',
    aggregation=pymortar.MAX,
    window='15m',
    timeseries=[
        pymortar.Timeseries(
            view='equip',
            dataVars=['?state']
        )
    ]
)

qr = client.qualify([meterview.definition, equipview.definition])
print(qr.sites)

req = pymortar.FetchRequest(
#     sites=qr.sites,
    sites=['ciee'],
    views=[meterview, equipview],
    dataFrames=[meterdf, equipdf],
    time=pymortar.TimeParams(
        start="2018-01-01T00:00:00Z",
        end="2019-01-01T00:00:00Z",
    )
)

INFO:root:Got event ChannelConnectivity.CONNECTING
INFO:root:Got event ChannelConnectivity.READY
INFO:root:Generating a new JWT token. Your old token may have expired


['berkeley-corporate-yard', 'south-berkeley-senior-center', 'hayward-station-8', 'avenal-animal-shelter', 'bwfp', 'north-berkeley-senior-center', 'avenal-recreation-center', 'rfs', 'avenal-veterans-hall', 'ciee', 'orinda-community-center', 'avenal-public-works-yard', 'thur', 'local-butcher-shop', 'hayward-station-1', 'avenal-movie-theatre', 'garber', 'orinda-public-library', 'jesse-turner-center', 'word-of-faith-cc']


In [4]:
resp = client.fetch(req)

dfs = []
for dfname in resp.dataFrames:
    df = resp[dfname]#.resample('15T').mean()
    df = df.interpolate().fillna(method='ffill').fillna(method='bfill')
    df.loc[:, 'timestamp'] = df.index
    df = pd.melt(df, id_vars=['timestamp'])
    
    df.loc[:, 'weekday'] = preprocessing.LabelEncoder().fit_transform(df['timestamp'].dt.weekday_name)
    df.loc[:, 'hour'] = df['timestamp'].dt.hour
    
    dfs.append(df.copy())

enc = preprocessing.LabelEncoder()
a = pd.concat(dfs)
groups = a.groupby('timestamp')
def fix_(g):
    g.pop('timestamp')
    g.loc[:, 'variable'] = enc.fit_transform(g.pop('variable').values)
    return g.values.flatten()
g = groups.apply(fix_)
m = pd.DataFrame.from_records(g.values)
m_ = m.set_index(g.index)

PREV=32
NEXT=32
m = m_.copy()
prevs = [m] + [m.shift(-n) for n in range(1, PREV+1)]
# map(lambda x: x.pop('timestamp'), prevs)
nexts = [m[[0]]] + [m[[0]].shift(n) for n in range(1, NEXT+1)]
m.pop(0)

_X = pd.concat(prevs, axis=1)
_Y = pd.concat(nexts, axis=1)

_Y.dropna(inplace=True)
_X.dropna(inplace=True)

Y = _Y.values
X = _X.values

/Users/johnyang/anaconda3/envs/autoML/lib/python3.6/site-packages/pandas/core/indexing.py:362: FutureWarning:

Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.



### Ridge Regression, PCA

In [5]:
split = int(.2 * len(Y))
trainX, trainY = X[:-split,], Y[:-split,]
testX, testY = X[-split:,], Y[-split:,]
trainX.shape, trainY.shape

((28006, 659), (28006, 33))

In [14]:
import numpy as np
import pandas as pd

# Sklearn Models
import sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Sklearn Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Place data in DataFrame objects
trainX_df = pd.DataFrame(trainX)
trainY_df = pd.DataFrame(trainY)
testX_df = pd.DataFrame(testX)
testY_df= pd.DataFrame(testY)

trainX_df.head()

,0,1,2,3,4,5,6,7,8,9,...,649,650,651,652,653,654,655,656,657,658
0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,8.0,3.0,0.0,1.0,8.0,2.0,0.0,1.0,8.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,8.0,3.0,0.0,1.0,8.0,2.0,0.0,1.0,8.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,8.0,3.0,0.0,1.0,8.0,2.0,0.0,1.0,8.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,4.0,0.0,1.0,0.0,...,8.0,3.0,0.0,1.0,8.0,2.0,0.0,1.0,8.0,0.0
4,1.0,1.0,1.0,0.0,1.0,1.0,4.0,0.0,1.0,1.0,...,9.0,3.0,0.0,1.0,9.0,2.0,0.0,1.0,9.0,0.0


In [12]:
trainY_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,2180.571429,2443.692308,2534.693878,2607.123288,2533.737374,2546.756757,2538.108108,2640.800000,2497.227723,2601.081081,...,2532.374101,2633.870968,2472.695652,2491.027027,2432.857143,2426.086957,2492.112676,2535.783784,2154.244604,2263.546099
1,2393.714286,2180.571429,2443.692308,2534.693878,2607.123288,2533.737374,2546.756757,2538.108108,2640.800000,2497.227723,...,2486.120219,2532.374101,2633.870968,2472.695652,2491.027027,2432.857143,2426.086957,2492.112676,2535.783784,2154.244604
2,2326.478873,2393.714286,2180.571429,2443.692308,2534.693878,2607.123288,2533.737374,2546.756757,2538.108108,2640.800000,...,2563.165468,2486.120219,2532.374101,2633.870968,2472.695652,2491.027027,2432.857143,2426.086957,2492.112676,2535.783784
3,2227.301587,2326.478873,2393.714286,2180.571429,2443.692308,2534.693878,2607.123288,2533.737374,2546.756757,2538.108108,...,2521.714286,2563.165468,2486.120219,2532.374101,2633.870968,2472.695652,2491.027027,2432.857143,2426.086957,2492.112676
4,2234.532374,2227.301587,2326.478873,2393.714286,2180.571429,2443.692308,2534.693878,2607.123288,2533.737374,2546.756757,...,2564.919786,2521.714286,2563.165468,2486.120219,2532.374101,2633.870968,2472.695652,2491.027027,2432.857143,2426.086957


In [16]:
# Setting up Pipelines
logistic = Ridge(max_iter=10000, tol=1e-5, random_state=0)
pca = PCA()
pipelines = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

param_grid = {
    'pca__n_components': [3, 5, 7, 9, 11, 13, 15, 19],
    'logistic__alpha': np.logspace(-4, 4, 5),
}

search = GridSearchCV(pipelines, param_grid, iid=False, cv=5)

search.fit(trainX, trainY)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None,
                                            random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('logistic',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True,
                                              max_iter=10000, normalize=False,
                                              random_state=0, solver='auto',
                                              tol=1e-05))],
                                verbose=False),
             iid=False, n_jobs=None,
             param_grid={'logis

In [18]:
search.best_params_

{'logistic__alpha': 100.0, 'pca__n_components': 19}

In [19]:
predictions = search.predict(testX)
mean_squared_error(predictions, testY)

177613.84008316122

### Polynomial

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly_features = PolynomialFeatures(degree=2)
x_poly = poly_features.fit_transform(trainX)

linear_model = LinearRegression()
linear_model.fit(x_poly, trainY)

y_poly_predictions = linear_model.predict(x_poly)

### Lasso Regression